### Hacer que eres un usuario

In [1]:
usuario_fake = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'es-ES, es;q=0.5'})

In [2]:
import requests
import bs4
import pandas as pd
import csv

In [3]:
#Pegar la url de amazon gorros
URL = "https://www.amazon.com/s?k=gorros&crid=22JB1LSDV7DJV&sprefix=gorros%2Caps%2C193&ref=nb_sb_noss_2"

In [4]:
#Entrar en URL como usuario usuario_fake
amazon = requests.get(URL, headers=usuario_fake)


In [5]:
amazon 

<Response [503]>

In [6]:
sopa = bs4.BeautifulSoup(amazon.content, features="lxml")


In [7]:
#Span es la separacion de los codigos que hay
sopa.findAll("span")

[]

# Scripts para Amazon


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv


def get_title(sopa):
    try:
        title = sopa.find("span", attrs={"id":'productTitle'}).string.strip()
    except AttributeError:
        return ""    
    return title

def get_price(sopa):
    try:
        price = sopa.find("span", attrs={'class':'a-offscreen'}).string.strip()
    except AttributeError:
        try:
            price = sopa.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()
        except:     
            return ""  
    return price

def get_rating(sopa):
    try:
        rating = sopa.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = sopa.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            return ""
    return rating

def get_review_count(sopa):
    try:
        review_count = sopa.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        return ""
    return review_count

def get_availability(sopa):
    try:
        available = sopa.find("div", attrs={'id':'availability'}).find("span").string.strip()
    except AttributeError:
        return "Not Available"
    return available

def main():
    HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36', 'Accept-Language': 'en-US'}
    URL = input(" ") #Código para que sea también desde la consola
    webpage = requests.get(URL, headers=HEADERS)
    sopa = BeautifulSoup(webpage.content, "lxml")

    links = sopa.find_all( "https://www.amazon.com/s?k=gorros&crid=22JB1LSDV7DJV&sprefix=gorros%2Caps%2C193&ref=nb_sb_noss_2", attrs={'class':'a-link-normal s-no-outline'})
    links_list = [link.get('href') for link in links]


    with open('amazon_product.csv', 'w', encoding="utf-8-sig") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Title', 'Price', 'Rating', 'ReviewCount', 'Availability'])

        for link in links_list:
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
            new_sopa = BeautifulSoup(new_webpage.content, "lxml")

            title = get_title(new_sopa)
            price = get_price(new_sopa)
            rating = get_rating(new_sopa)
            review_count = get_review_count(new_sopa)
            availability = get_availability(new_sopa)

            print("Product Title =", title)
            print("Product Price =", price)
            print("Product Rating =", rating)
            print("Number of Product Reviews =", review_count)
            print("Availability =", availability)
            print('\n')

            # Escribe la información del producto en el archivo CSV.
            csv_writer.writerow([title, price, rating, review_count, availability])

if __name__ == '__main__':
    main()

In [11]:
gorros = pd.read_csv(r"C:\Users\34650\Desktop\BOOTCAMP_CURSO\Temario\MODULO 1\Web Scrapping\Amazon\amazon_product.csv")
